In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap

# import geopandas as gpd
# from shapely.geometry  import Point
# from shapely.geometry import shape
# import seaborn as sns
from matplotlib.patches import Rectangle
import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape


import pandas as pd 


# defs

## other functions

In [ ]:
def create_mask(fo,lat_obs_cru,lon_obs_cru,buffer=0,plot=0):
  filename=gpd.read_file(fo)
  shp_geom = shape(filename.geometry[0])

  xxc, yyc = meshgrid(lon_obs_cru,lat_obs_cru)
  mask_shape=np.zeros(xxc.shape,dtype=bool)
  if plot==1:
    x,y = shp_geom.exterior.xy
    plt.plot(x,y)
  N1=xxc.shape[0]
  N2=xxc.shape[1]

  for lat_i in range(0,N1):
    for lon_i in range(0,N2):
      xi=xxc[lat_i,lon_i]
      yi=yyc[lat_i,lon_i]
      m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))
      mask_shape[lat_i,lon_i]=not m1

  return mask_shape

# init

In [ ]:
Vars = ["US_T","US_P","IND_T","IND_P"]
gcm_list = ['GFDL_CM3','CanESM2','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']



S_scores_random = {}  # Vars x N_randoms
S_scores_ESMs = {}  # Vars x N_randoms



In [ ]:
def S_score_2D(data_ref, data_model,mask_inf):
    # mask is true for exterior values
    data_1 = data_ref
    data_2 = data_model 
    ind_1 = np.isnan(data_1)
    ind_2 = np.isnan(data_2)
    data_1[ind_2] = np.nan
    data_2[ind_1] = np.nan
    data_1[mask_inf == True] = np.nan
    data_2[mask_inf == True] = np.nan

    data_1 = data_1.flatten()
    data_2 = data_2.flatten()
    
    data_1 =  np.delete(data_1,np.isnan(data_1))
    data_2 =  np.delete(data_2,np.isnan(data_2))
    R = np.corrcoef(data_1,data_2)[0,1]

    SDR = np.std(data_2)/np.std(data_1)

    S_score = (1+R)**4  / ( 4 * (SDR + 1 / SDR)**2 )
    
    
    return S_score

In [ ]:
from tqdm import tqdm

## India Bias 

In [ ]:
gcm_name = 'CanESM2'
fig_name_suffix = '' # _eqm
var = "IND_P"
map_bounds=[65, 40, 100, 5]

path_out ='/home/vikram/BCSD_data/India_P_BCSDd' +fig_name_suffix
imd=np.load(path_out + '/data_obs_imd_pred.npy')
gcm_meta_01=(np.load(path_out + '/bcsd_imd_metadata_'+gcm_name+'.npy',
                     allow_pickle=True))
lat_obs_in=gcm_meta_01.item()['lat_obs']
lon_obs_in=gcm_meta_01.item()['lon_obs']
mask_inf=create_mask('/home/vikram/BCSD_Shapefiles/ind.shp',lat_obs_in.data,lon_obs_in.data,buffer=1,plot=0)



# out_id='pre_'+gcm_name
# bcsd_gcm_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_'+out_id+'.npy'))

# rnd_itr = 10
# bcsd_rnd_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_pre_rnd_new_'+str(rnd_itr)+'.npy'))





S_scores_random[var] = []
S_scores_ESMs[var] = []

obs_ind  = np.mean(imd,axis= 0)

for rnd_itr in tqdm(range(10)):
    bcsd_rnd_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_pre_rnd_new_'+str(rnd_itr)+'.npy'))
    rnd_ind = np.mean(bcsd_rnd_01[150::365,:,:],axis=0)
    S_scores_random[var].append(S_score_2D(obs_ind,rnd_ind,mask_inf))
    
    
for gcm_name in gcm_list:
    out_id='pre_'+gcm_name 
    bcsd_gcm_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_'+out_id+'.npy'))
    bcsd_ind = np.mean(bcsd_gcm_01[150::365,:,:],axis= 0)
    S_scores_ESMs[var].append(S_score_2D(obs_ind,bcsd_ind,mask_inf))
        

## India Bias temp 

In [ ]:

path_out ='/home/vikram/BCSD_data/India_T_BCSDd'+fig_name_suffix
imd=uu.correct_nan_inf_neg(np.load(path_out + '/data_obs_imd_pred.npy'))
gcm_meta_01=(np.load(path_out + '/bcsd_imd_metadata_'+gcm_name+'.npy',
                     allow_pickle=True))
lat_obs_in_t=gcm_meta_01.item()['lat_obs']
lon_obs_in_t=gcm_meta_01.item()['lon_obs']

# out_id='tas_'+gcm_name
# bcsd_gcm_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_'+out_id+'.npy'))


# rnd_itr = 10
# bcsd_rnd_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_tas_rnd_new_'+str(rnd_itr)+'.npy'))



mask_inf_temp=create_mask('/home/vikram/BCSD_Shapefiles/ind.shp',lat_obs_in_t.data,lon_obs_in_t.data,buffer=1,plot=0)

var = "IND_T"
S_scores_random[var] = []
S_scores_ESMs[var] = []

obs_ind  = np.mean(imd,axis= 0)

for rnd_itr in tqdm(range(10)):
    bcsd_rnd_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_tas_rnd_new_'+str(rnd_itr)+'.npy'))
    rnd_ind = np.mean(bcsd_rnd_01,axis=0)
    
    S_scores_random[var] .append(S_score_2D(obs_ind,rnd_ind,mask_inf_temp))
    
    
for gcm_name in gcm_list:
    out_id='tas_'+gcm_name 
    bcsd_gcm_01=uu.correct_nan_inf_neg(np.load(path_out + '/BCSD_outputs_'+out_id+'.npy'))
    bcsd_ind = np.mean(bcsd_gcm_01,axis= 0)
    
    S_scores_ESMs[var].append(S_score_2D(obs_ind,bcsd_ind,mask_inf_temp))
        

## Bias  us

In [ ]:
path_US_T='/home/vikram/BCSD_data/US_T_BCSD'+fig_name_suffix
path_US_P='/home/vikram/BCSD_data/US_P_BCSD'+fig_name_suffix

metadata=np.load(path_US_P+'/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
lat_obs_us=metadata.item()['lat_obs']
lon_obs_us=metadata.item()['lon_obs']
mask_usf=create_mask('/home/vikram/BCSD_Shapefiles/us_rough.shp',lat_obs_us.data,lon_obs_us.data-360,buffer=1,plot=0)

pre_obs=np.load(path_US_P+'/pre_obs.npy')
# pre_bcsd=np.load(path_US_P+'/BCSD_outputs_pre_'+gcm_names[gcm_iter]+'_cru.npy')

    
var = "US_P"
S_scores_random[var] = []
S_scores_ESMs[var] = []

obs_ind  = pre_obs.mean(axis = 0)

for rnd_itr in tqdm(range(10)):
    pre_rnd=np.load(path_US_P+'/BCSD_outputs_pre_rnd_new2_'  + str(rnd_itr) + '_cru.npy')
    rnd_ind = np.mean(pre_rnd,axis=0)
    
    S_scores_random[var] .append(S_score_2D(obs_ind,rnd_ind,mask_usf))
    
plt.figure(figsize = (20,20))

for giter,gcm_name in enumerate(gcm_list):
    pre_bcsd=np.load(path_US_P+'/BCSD_outputs_pre_'+gcm_name+'_cru.npy')
    bcsd_ind = np.mean(pre_bcsd[::12,:,:],axis= 0)
    S_scores_ESMs[var].append(S_score_2D(obs_ind,bcsd_ind,mask_usf))
#     plt.subplot(5,5,giter+1)
#     plt.pcolor(bcsd_ind)
    
    

In [ ]:
for giter,gcm_name in enumerate(gcm_list):
    pre_bcsd=np.load(path_US_P+'/BCSD_outputs_pre_'+gcm_name+'_cru.npy')
    bcsd_ind = np.mean(pre_bcsd,axis= 0)
    S_scores_ESMs[var].append(S_score_2D(obs_ind,bcsd_ind,mask_usf))
    # plt.subplot(5,5,giter+1)
    plt.plot(pre_bcsd[:36,25,50])
    print(np.mean(pre_bcsd[:,25,50]))
plt.figure()
for rnd_itr in tqdm(range(10)):
    pre_rnd=np.load(path_US_P+'/BCSD_outputs_pre_rnd_new2_'  + str(rnd_itr) + '_cru.npy')
    rnd_ind = np.mean(pre_rnd,axis=0)
    
    S_scores_random[var] .append(S_score_2D(obs_ind,rnd_ind,mask_usf))
    

    plt.plot(pre_rnd[:36,25,50])
    print(np.mean(pre_rnd[:,25,50]))
    

In [ ]:
bcsd_ind.shape

## Bias  us tmp

In [ ]:
tas_obs=np.load(path_US_T + '/tas_obs.npy')

var = "US_T"
S_scores_random[var] = []
S_scores_ESMs[var] = []

obs_ind  = tas_obs.mean(axis = 0)

for rnd_itr in tqdm(range(10)):
    tas_rnd=np.load(path_US_T+'/BCSD_outputs_tas_rnd_new2_'  + str(rnd_itr) + '_cru.npy')
    rnd_ind = np.mean(tas_rnd,axis=0)
    S_scores_random[var].append(S_score_2D(obs_ind,rnd_ind,mask_usf))
    
    
for gcm_name in gcm_list:
    tas_bcsd=np.load(path_US_T + '/BCSD_outputs_tas_'+gcm_name+'_cru.npy')
    bcsd_ind = np.mean(tas_bcsd,axis= 0)
    S_scores_ESMs[var].append(S_score_2D(obs_ind,bcsd_ind,mask_usf))


In [ ]:
plt.boxplot( list(S_scores_random[var] for var in Vars))
plt.xticks(np.arange(4)+1, Vars)

markers=['o','^','s','p','P','*','h','+','x','X','D']
N_esm =len(gcm_list)
for itr,var in enumerate(Vars):
    for gi in range(N_esm):
        plt.plot(itr +1 ,S_scores_ESMs[var][gi],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
plt.ylim([0,1.1])

    

In [ ]:
ss_bcsd = S_score_2D(tas_obs.mean(axis = 0),tas_bcsd.mean(axis = 0),mask_usf)    

plt.boxplot( [S_scores_us_tas,S_scores_us_pre])
plt.plot( 1, ss_bcsd,'*')

ss_bcsd = S_score_2D(pre_obs.mean(axis = 0),pre_bcsd.mean(axis = 0),mask_usf)    

# plt.hist( 2, S_scores_us_pre)
plt.plot( 2, ss_bcsd,'*')

plt.ylim(0,1.1)

In [ ]:
S_scores_us_tas

In [ ]:
S_scores_us_tas

# defs


In [ ]:
def S_score_2D(data_ref, data_model,mask_inf):
    # mask is true for exterior values
    data_1 = data_ref
    data_2 = data_model 
    ind_1 = np.isnan(data_1)
    ind_2 = np.isnan(data_2)
    data_1[ind_2] = np.nan
    data_2[ind_1] = np.nan
    data_1[mask_inf == True] = np.nan
    data_2[mask_inf == True] = np.nan

    data_1 = data_1.flatten()
    data_2 = data_2.flatten()
    
    data_1 =  np.delete(data_1,np.isnan(data_1))
    data_2 =  np.delete(data_2,np.isnan(data_2))
    R = np.corrcoef(data_1,data_2)[0,1]

    SDR = np.std(data_2)/np.std(data_1)

    S_score = (1+R)**4  / ( 4 * (SDR + 1 / SDR)**2 )
    
    
    return S_score

In [ ]:
data_in_pre[3].shape

In [ ]:
import scipy.stats as stats
data_group1 = data_us_pre[2]
data_group2 = data_us_pre[3]


_,pval = stats.ttest_ind(data_group1,data_group2,
                                 axis=0, equal_var = False)

plt.hist(pval.flatten())

In [ ]:
plt.plot(data_group1[::12,25,60])
plt.plot(data_group2[::12,25,60])
stats.ttest_ind(data_group1[:,25,60],data_group2[:,25,60])


In [ ]:
import scipy.stats as stats
data_group1 = data_in_pre[2].reshape([365,-1,33,35],order='F').mean(axis=0)
data_group2 = data_in_pre[3].reshape([365,-1,33,35],order='F').mean(axis=0)


_,pval = stats.ttest_ind(data_group1,data_group2,
                                 axis=0, equal_var = False)



lat_box=uu.cen_to_box(lat_obs_in)
lon_box=uu.cen_to_box(lon_obs_in)
mp_spacing=10
map_bounds=[65,37.5,100,7.5]
shape_bound='/home/vikram/BCSD_Shapefiles/ind'

        
        
map = Basemap(projection='cyl',resolution='l',
              llcrnrlon=map_bounds[0], 
              urcrnrlat=map_bounds[1],
              urcrnrlon=map_bounds[2], 
              llcrnrlat=map_bounds[3])

xx, yy = meshgrid(lon_obs_in,lat_obs_in )
map.pcolor(xx, yy, data_group1.mean(axis=0))
sig=plt.gca().contourf(xx,yy,pval,
            cmap=None,extend='both',levels=[0.05,1],hatches=['x',None],
            colors='none')  


map.readshapefile(shape_bound,'DD',drawbounds=True)

    
    
# plt.figure()
# plt.pcolor(pval<10.05)
# plt.colorbar()
# plt.pcolor(pval,cmap ='Reds',vmax= 0.05,vmin = 0)
# plt.colorbar()

In [ ]:
data_group1.shape

In [ ]:
plt.hist(data_group1[:,later_us,loner_us])
plt.hist(data_group2[:,later_us,loner_us])

In [ ]:
plt.hist(data_group1[:,later_us,loner_us])
plt.hist(data_group2[:,later_us,loner_us])

In [ ]:
plt.hist(pval.flatten())

In [ ]:
data_plot=[data_us_tas,data_us_pre,data_in_tas,data_in_pre]
titles_plot=['Temperature','Precipitation','Temperature','Precipitation']
clab_plot=['$\degree$C','mm/month','$\degree$C','mm/day']
masks=[mask_usf,mask_usf,mask_inf_temp,mask_inf]
clims=[[-4,30],[0,150],[15,30],[0,12]]
ext=['both','max','both','max']
cmaps = ['RdBu_r','BrBG','RdBu_r','BrBG']
fig=plt.figure(figsize=(14,12))
for sitery in range(4):
    if sitery<2:
      lat_box=uu.cen_to_box(lat_obs_us)
      lon_box=uu.cen_to_box(lon_obs_us)
      mp_spacing=10
      map_bounds=[360-125,52.5,360-70,22.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/us_rough'
      later_us=25
      loner_us=52
      rhx=1#lon_obs_us[loner_us+1]-lon_obs_us[loner_us]
      rhy=1#lat_obs_us[later_us+1]-lat_obs_us[later_us]
      ll=(lon_obs_us[loner_us]-0.5*rhx,lat_obs_us[later_us]-rhy*0.5)  
      r_fac = 12
      day_r =0
    
    elif sitery == 3:
      lat_box=uu.cen_to_box(lat_obs_in)
      lon_box=uu.cen_to_box(lon_obs_in)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=14
      loner=16
      lat_obs_in[later]
      lon_obs_in[loner]
      rhx=lon_obs_in[loner+1]-lon_obs_in[loner]
      rhy=lat_obs_in[later+1]-lat_obs_in[later]
      ll=(lon_obs_in[loner]-0.5*rhx,lat_obs_in[later]-rhy*0.5)
      r_fac = 365
      day_r =170
    elif sitery == 2:
      lat_box=uu.cen_to_box(lat_obs_in_t)
      lon_box=uu.cen_to_box(lon_obs_in_t)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=14
      loner=16
      lat_obs_in_t[later]
      lon_obs_in_t[loner]
      rhx=lon_obs_in_t[loner+1]-lon_obs_in_t[loner]
      rhy=lat_obs_in_t[later+1]-lat_obs_in_t[later]
      ll=(lon_obs_in_t[loner]-0.5*rhx,lat_obs_in_t[later]-rhy*0.5)
      r_fac = 365
      day_r =170
    else:
      pass


    ax=plt.subplot(4,1,sitery+1)
    siter=sitery
    
    if sitery == 0 or sitery == 2:
        data_x=ma.MaskedArray(np.mean(data_plot[sitery][2]-273.15,axis=0),mask=masks[sitery])
    else:
        data_x=ma.MaskedArray(np.mean(data_plot[sitery][2],axis=0),mask=masks[sitery])

    map = Basemap(projection='cyl',resolution='l',
                  llcrnrlon=map_bounds[0], 
                  urcrnrlat=map_bounds[1],
                  urcrnrlon=map_bounds[2], 
                  llcrnrlat=map_bounds[3])

    xx, yy = meshgrid(lon_box,lat_box )
    
    map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,1,0,0], linewidth=1.0) #longitudes
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
    
    if siter<8:
      map.drawcountries(linewidth=1)
      map.drawcoastlines(linewidth=1)
    else:
      map.readshapefile(shape_bound,'DD',drawbounds=True)

    data_group1 = data_plot[sitery][2].reshape([r_fac,-1,data_x.shape[0],data_x.shape[1]],order='F')[day_r]
    data_group2 = data_plot[sitery][3].reshape([r_fac,-1,data_x.shape[0],data_x.shape[1]],order='F')[day_r]

    _,pval = stats.ttest_ind(data_group1,data_group2,
                                 axis=0, equal_var = False)
    map.pcolor(xx, yy, np.std(data_group1,axis= 0)-np.std(data_group2,axis= 0))#,vmax = 0.05, vmin = 0)
    map.colorbar()
    xx1, yy1 = meshgrid(uu.box_to_cen(lon_box),uu.box_to_cen(lat_box) )
    sig=plt.gca().contourf(xx1,yy1,pval,
            cmap=None,extend='both',levels=[0.05,1],hatches=['x',None],
            colors='none')  

plt.tight_layout()


# Plot Mean figure

In [ ]:
data_plot=[data_us_tas,data_us_pre,data_in_tas,data_in_pre]
titles_plot=['Temperature','Precipitation','Temperature','Precipitation']
clab_plot=['$\degree$C','mm/month','$\degree$C','mm/day']
masks=[mask_usf,mask_usf,mask_inf_temp,mask_inf]
clims=[[-4,30],[0,150],[15,30],[0,12]]
ext=['both','max','both','max']
cmaps = ['RdBu_r','BrBG','RdBu_r','BrBG']
fig=plt.figure(figsize=(14,12))
for sitery in range(4):
  for siterx in range(4):

    if sitery<2:
      lat_box=uu.cen_to_box(lat_obs_us)
      lon_box=uu.cen_to_box(lon_obs_us)
      mp_spacing=10
      map_bounds=[360-125,52.5,360-70,22.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/us_rough'
      later_us=25
      loner_us=52
      rhx=1#lon_obs_us[loner_us+1]-lon_obs_us[loner_us]
      rhy=1#lat_obs_us[later_us+1]-lat_obs_us[later_us]
      ll=(lon_obs_us[loner_us]-0.5*rhx,lat_obs_us[later_us]-rhy*0.5)  
      r_fac = 12
    elif sitery == 3:
      lat_box=uu.cen_to_box(lat_obs_in)
      lon_box=uu.cen_to_box(lon_obs_in)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=14
      loner=16
      lat_obs_in[later]
      lon_obs_in[loner]
      rhx=lon_obs_in[loner+1]-lon_obs_in[loner]
      rhy=lat_obs_in[later+1]-lat_obs_in[later]
      ll=(lon_obs_in[loner]-0.5*rhx,lat_obs_in[later]-rhy*0.5)
      r_fac = 365
    elif sitery == 2:
      lat_box=uu.cen_to_box(lat_obs_in_t)
      lon_box=uu.cen_to_box(lon_obs_in_t)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=14
      loner=16
      lat_obs_in_t[later]
      lon_obs_in_t[loner]
      rhx=lon_obs_in_t[loner+1]-lon_obs_in_t[loner]
      rhy=lat_obs_in_t[later+1]-lat_obs_in_t[later]
      ll=(lon_obs_in_t[loner]-0.5*rhx,lat_obs_in_t[later]-rhy*0.5)
      r_fac = 365
    else:
      pass


    ax=plt.subplot(4,4,siterx*4+sitery+1)
    siter=sitery*4+siterx
    # print(siter)
    if sitery == 0 or sitery == 2:
        data_x=ma.MaskedArray(np.mean(data_plot[sitery][siterx]-273.15,axis=0),mask=masks[sitery])
    else:
        data_x=ma.MaskedArray(np.mean(data_plot[sitery][siterx],axis=0),mask=masks[sitery])

    map = Basemap(projection='cyl',resolution='l',
                  llcrnrlon=map_bounds[0], 
                  urcrnrlat=map_bounds[1],
                  urcrnrlon=map_bounds[2], 
                  llcrnrlat=map_bounds[3])

    xx, yy = meshgrid(lon_box,lat_box )
    map.pcolor(xx, yy, data_x,cmap=cmaps[sitery])
    map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,1,0,0], linewidth=1.0) #longitudes
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
    cbar = map.colorbar(extend=ext[sitery],location='bottom')
    cbar.set_label(clab_plot[sitery], rotation=0)
    plt.clim(clims[sitery])
    if siter<8:
      map.drawcountries(linewidth=1)
      map.drawcoastlines(linewidth=1)
    else:
      map.readshapefile(shape_bound,'DD',drawbounds=True)

    data_group1 = data_in_pre[2].reshape([365,-1,33,35],order='F').mean(axis=0)
    data_group2 = data_in_pre[3].reshape([365,-1,33,35],order='F').mean(axis=0)


    _,pval = stats.ttest_ind(data_group1,data_group2,
                                 axis=0, equal_var = False)



    rect1 = Rectangle(ll, rhx, rhy, facecolor='none',edgecolor='black' )
    pt1=ax.add_patch(rect1) 
    pt1.zorder=1
    pt1.set_lw(2)
    lxx=0.0
    lyy=1.0
    plt.text(lxx, lyy, chr(siterx*4+sitery+96+1), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=800,
            fontsize=24,color='k')
    
fig.text(0.0, 0.875, 'Observation', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.625, 'ESM', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.375, 'BCSD-ESM', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.125, 'BCSD-Rnd', va='center', ha='center', rotation='vertical', fontsize=12)


fig.text( 0.875,1, 'India Daily Precipitation', va='center', ha='center', fontsize=12)
fig.text( 0.625,1, 'India Daily Temperature', va='center', ha='center',  fontsize=12)
fig.text( 0.375,1, 'US Monthly Precipitation', va='center', ha='center',  fontsize=12)
fig.text( 0.125,1, 'US Monthly Temperature', va='center', ha='center',  fontsize=12)

    #map.drawcoastlines(linewidth=1)
    #plt.title(titles_plot[siter],fontsize=20)
fig.suptitle(gcm_name)
plt.tight_layout()
# plt.savefig('./Figures/Figure2/mean_maps_'+gcm_name+'_'+'gamma'+fig_name_suffix+'.svg')


## S Score


In [ ]:
D=4
N_index= 4
S_score_matrix = np.zeros((N_index,D))


data_plot=[data_us_tas,data_us_pre,data_in_tas,data_in_pre]
masks=[mask_usf,mask_usf,mask_inf_temp,mask_inf]

for itr in range(4):
    for d in range(D):
        S_score_matrix[itr,d] = (S_score_2D(np.mean(data_plot[itr],axis=1)[0,:,:],np.mean(data_plot[itr],axis=1)[d,:,:],masks[itr]))
        
        


import seaborn as sns

h=sns.heatmap(S_score_matrix , linewidth = 2 , cmap = 'Reds', 
          annot=True,
          fmt=".2f", square=True,
          xticklabels=['O','G','B','R'],
          yticklabels=['US - T','US - P','In - T','In - P'],vmin=0,vmax=1,
             
          cbar_kws={'label': 'S score','extend':None},cbar = True)
plt.savefig('./Figures/Figure2/mean_maps_score_'+gcm_name+'_'+'gamma'+fig_name_suffix+'.svg')


# Plot Std figure

In [ ]:
data_plot=[data_us_tas,data_us_pre,data_in_tas,data_in_pre]
titles_plot=['Temperature','Precipitation','Temperature','Precipitation']
clab_plot=['$\degree$C','mm/month','$\degree$C','mm/day']
masks=[mask_usf,mask_usf,mask_inf_temp,mask_inf]
clims=[[0,15],[0,100],[0,10],[0,10]]
ext=['max','max','max','max']

fig=plt.figure(figsize=(14,12))
for sitery in range(4):
  for siterx in range(4):

    if sitery<2:
      lat_box=uu.cen_to_box(lat_obs_us)
      lon_box=uu.cen_to_box(lon_obs_us)
      mp_spacing=10
      map_bounds=[360-125,52.5,360-70,22.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/us_rough'
      later_us=25
      loner_us=50
      rhx=1#lon_obs_us[loner_us+1]-lon_obs_us[loner_us]
      rhy=1#lat_obs_us[later_us+1]-lat_obs_us[later_us]
      ll=(lon_obs_us[loner_us]-0.5*rhx,lat_obs_us[later_us]-rhy*0.5)    
    elif sitery == 3:
      lat_box=uu.cen_to_box(lat_obs_in)
      lon_box=uu.cen_to_box(lon_obs_in)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=15
      loner=12
      lat_obs_in[later]
      lon_obs_in[loner]
      rhx=lon_obs_in[loner+1]-lon_obs_in[loner]
      rhy=lat_obs_in[later+1]-lat_obs_in[later]
      ll=(lon_obs_in[loner]-0.5*rhx,lat_obs_in[later]-rhy*0.5)
    elif sitery == 2:
      lat_box=uu.cen_to_box(lat_obs_in_t)
      lon_box=uu.cen_to_box(lon_obs_in_t)
      mp_spacing=10
      map_bounds=[65,37.5,100,7.5]
      shape_bound='/home/vikram/BCSD_Shapefiles/ind'
      later=15
      loner=12
      lat_obs_in[later]
      lon_obs_in[loner]
      rhx=lat_obs_in_t[loner+1]-lon_obs_in_t[loner]
      rhy=lat_obs_in_t[later+1]-lon_obs_in_t[later]
      ll=(lon_obs_in_t[loner]-0.5*rhx,lat_obs_in_t[later]-rhy*0.5)
    else:
      pass


    ax=plt.subplot(4,4,siterx*4+sitery+1)
    siter=sitery*4+siterx
    # print(siter)
    if sitery == 0 or sitery == 2:
        data_x=ma.MaskedArray(np.std(data_plot[sitery][siterx]-273.15,axis=0),mask=masks[sitery])
    else:
        data_x=ma.MaskedArray(np.std(data_plot[sitery][siterx],axis=0),mask=masks[sitery])

    map = Basemap(projection='cyl',resolution='l',
                  llcrnrlon=map_bounds[0], 
                  urcrnrlat=map_bounds[1],
                  urcrnrlon=map_bounds[2], 
                  llcrnrlat=map_bounds[3])

    xx, yy = meshgrid(lon_box,lat_box )
    map.pcolor(xx, yy, data_x,cmap='RdBu_r')
    map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,1,0,0], linewidth=1.0) #longitudes
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
    cbar = map.colorbar(extend=ext[sitery],location='bottom')
    cbar.set_label(clab_plot[sitery], rotation=0)
    plt.clim(clims[sitery])
    if siter<8:
      map.drawcountries(linewidth=1)
      map.drawcoastlines(linewidth=1)
    else:
      map.readshapefile(shape_bound,'DD',drawbounds=True)

    rect1 = Rectangle(ll, rhx, rhy, facecolor='none',edgecolor='black' )
    # pt1=ax.add_patch(rect1) 
    pt1.zorder=1
    pt1.set_lw(2)
    lxx=0.0
    lyy=1.0
    plt.text(lxx, lyy, chr(siterx*4+sitery+96+1), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=800,
            fontsize=24,color='k')
    
fig.text(0.0, 0.875, 'Observation', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.625, 'ESM', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.375, 'BCSD-ESM', va='center', ha='center', rotation='vertical', fontsize=12)
fig.text(0.0, 0.125, 'BCSD-Rnd', va='center', ha='center', rotation='vertical', fontsize=12)


fig.text( 0.875,1, 'India Daily Precipitation', va='center', ha='center', fontsize=12)
fig.text( 0.625,1, 'India Daily Temperature', va='center', ha='center',  fontsize=12)
fig.text( 0.375,1, 'US Monthly Precipitation', va='center', ha='center',  fontsize=12)
fig.text( 0.125,1, 'US Monthly Temperature', va='center', ha='center',  fontsize=12)

    #map.drawcoastlines(linewidth=1)
    #plt.title(titles_plot[siter],fontsize=20)
fig.suptitle(gcm_name)
plt.tight_layout()
plt.savefig('./Figures/Figure2/std_maps_'+gcm_name+'_'+'gamma'+fig_name_suffix+'.svg')


In [ ]:
mask_tt = 1.0 - mask_inf_temp 
mask_tt[mask_tt==0] = np.nan

plt.pcolor(np.var(data_in_tas[2],axis=0)*mask_tt)
plt.colorbar()
# mask_tt

## S Score


In [ ]:
D=4
N_index= 4
S_score_matrix = np.zeros((N_index,D))


data_plot=[data_us_tas,data_us_pre,data_in_tas,data_in_pre]
masks=[mask_usf,mask_usf,mask_inf_temp,mask_inf]

for itr in range(4):
    for d in range(D):
        S_score_matrix[itr,d] = (S_score_2D(np.std(data_plot[itr],axis=1)[0,:,:],np.std(data_plot[itr],axis=1)[d,:,:],masks[itr]))
        
        


In [ ]:
import seaborn as sns

h=sns.heatmap(S_score_matrix , linewidth = 2 , cmap = 'Reds', 
          annot=True,
          fmt=".2f", square=True,
          xticklabels=['O','G','B','R'],
          yticklabels=['US - T','US - P','In - T','In - P'],vmin=0,vmax=1,
             
          cbar_kws={'label': 'S score','extend':None},cbar = True)
plt.savefig('./Figures/Figure2/std_maps_score_'+gcm_name+'_'+'gamma'+fig_name_suffix+'.svg')


# Plot time series

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
fig=plt.figure(figsize=(14,2.5))

x=data_us_tas2
ax=plt.subplot(141)
plt.plot(x[0])
plt.plot(x[1])
plt.plot(x[2]+1)
plt.plot(x[3])
plt.ylabel("$\degree$C")
plt.xlabel("Months")
plt.xticks(range(12),labels=['J','F','M','A','M','J','J','A','S','O','N','D'])
#plt.legend(['Obs','GCM','BCSD','BCSD-rnd'])
lxx=0.0
lyy=1.0
plt.text(lxx, lyy, chr(16+96+1), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=800,
        fontsize=24,color='k')

x=data_us_pre2
ax=plt.subplot(142)
plt.plot(x[0])
plt.plot(x[1])
plt.plot(x[2]+1)
plt.plot(x[3])
plt.ylabel("mm/month")
plt.xlabel("Months")
#plt.legend(['Obs','GCM','BCSD','BCSD-rnd'])
plt.xticks(range(12),labels=['J','F','M','A','M','J','J','A','S','O','N','D'])
lxx=0.0
lyy=1.0
plt.text(lxx, lyy, chr(17+96+1), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=800,
        fontsize=24,color='k')


x=data_in_tas2
ax=plt.subplot(143)
ww=10
plt.plot(moving_average(x[0]-273.15,ww))
plt.plot(moving_average(x[1]-273.15,ww))
plt.plot(moving_average(x[2]-273.15,ww))
plt.plot(moving_average(x[3]-273.15,ww))
plt.ylabel("$\degree$C")
plt.xlabel("Day of year")
lxx=0.0
lyy=1.0
plt.text(lxx, lyy, chr(18+96+1), horizontalalignment='right',verticalalignment='bottom', 
         transform=ax.transAxes,fontweight=800,
        fontsize=24,color='k')



x=data_in_pre2
ax=plt.subplot(144)
ww=10
plt.plot(moving_average(x[0],ww))
plt.plot(moving_average(x[1],ww))
plt.plot(moving_average(x[2],ww))
plt.plot(moving_average(x[3],ww))
plt.ylabel("mm/day")
plt.xlabel("Day of year")
plt.legend(['Obs','ESM','BCSD-ESM','BCSD-rnd'])
lxx=0.0
lyy=1.0
plt.text(lxx, lyy, chr(18+96+2), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=800,
        fontsize=24,color='k')
fig.suptitle(gcm_name)


plt.savefig('./Figures/Figure2/mean_ts_'+gcm_name+'_gamma'+fig_name_suffix+'.svg')
plt.show()

# QQplot


In [ ]:
import numbers

import numpy as np
import matplotlib.pyplot as plt


def qqplot(x, y, quantiles=None, interpolation='nearest', ax=None, rug=False,
           rug_length=0.05, rug_kwargs=None, **kwargs):
    """Draw a quantile-quantile plot for `x` versus `y`.

    Parameters
    ----------
    x, y : array-like
        One-dimensional numeric arrays.

    ax : matplotlib.axes.Axes, optional
        Axes on which to plot. If not provided, the current axes will be used.

    quantiles : int or array-like, optional
        Quantiles to include in the plot. This can be an array of quantiles, in
        which case only the specified quantiles of `x` and `y` will be plotted.
        If this is an int `n`, then the quantiles will be `n` evenly spaced
        points between 0 and 1. If this is None, then `min(len(x), len(y))`
        evenly spaced quantiles between 0 and 1 will be computed.

    interpolation : {‘linear’, ‘lower’, ‘higher’, ‘midpoint’, ‘nearest’}
        Specify the interpolation method used to find quantiles when `quantiles`
        is an int or None. See the documentation for numpy.quantile().

    rug : bool, optional
        If True, draw a rug plot representing both samples on the horizontal and
        vertical axes. If False, no rug plot is drawn.

    rug_length : float in [0, 1], optional
        Specifies the length of the rug plot lines as a fraction of the total
        vertical or horizontal length.

    rug_kwargs : dict of keyword arguments
        Keyword arguments to pass to matplotlib.axes.Axes.axvline() and
        matplotlib.axes.Axes.axhline() when drawing rug plots.

    kwargs : dict of keyword arguments
        Keyword arguments to pass to matplotlib.axes.Axes.scatter() when drawing
        the q-q plot.
    """
    # Get current axes if none are provided
    if ax is None:
        ax = plt.gca()

    if quantiles is None:
        quantiles = min(len(x), len(y))

    # Compute quantiles of the two samples
    if isinstance(quantiles, numbers.Integral):
        quantiles = np.linspace(start=0, stop=1, num=int(quantiles))
    else:
        quantiles = np.atleast_1d(np.sort(quantiles))
    x_quantiles = np.quantile(x, quantiles, interpolation=interpolation)
    y_quantiles = np.quantile(y, quantiles, interpolation=interpolation)

    # Draw the rug plots if requested
    if rug:
        # Default rug plot settings
        rug_x_params = dict(ymin=0, ymax=rug_length, c='gray', alpha=0.5)
        rug_y_params = dict(xmin=0, xmax=rug_length, c='gray', alpha=0.5)

        # Override default setting by any user-specified settings
        if rug_kwargs is not None:
            rug_x_params.update(rug_kwargs)
            rug_y_params.update(rug_kwargs)

        # Draw the rug plots
        for point in x:
            ax.axvline(point, **rug_x_params)
        for point in y:
            ax.axhline(point, **rug_y_params)

    # Draw the q-q plot
    ax.scatter(x_quantiles, y_quantiles, **kwargs)


In [ ]:
plt.figure()
ax= plt.subplot(111)
x=data_qq_ip
qqplot(x[1],x[0], c='r', alpha=0.5, edgecolor='k',ax=ax)
qqplot(x[2],x[0],  c='g', alpha=0.5, edgecolor='k',ax=ax)
qqplot( x[3],x[0], c='b', alpha=0.5, edgecolor='k',ax=ax)
plt.ylim([0,80])
plt.xlim([0,80])
plt.xlabel('mm/day')
plt.ylabel('Observed data')
plt.legend(['GCM','BCSD-GCM','BCSD-Rnd'])
plt.plot([0,60],[0,60])
plt.axes().set_aspect('equal')
plt.savefig('./Figures/Figure2/qq_plot_ip'+gcm_name+'.svg')


plt.figure()
x=data_qq_ut
qqplot(x[1],x[0], c='r', alpha=0.5, edgecolor='k')
qqplot(x[2],x[0],  c='g', alpha=0.5, edgecolor='k')
qqplot( x[3],x[0], c='b', alpha=0.5, edgecolor='k')
ll=6
plt.ylim([-ll,ll+2])
plt.xlim([-ll,ll+2])

plt.xlabel('°C')
plt.ylabel('Observed data')
plt.legend(['GCM','BCSD-GCM','BCSD-Rnd'])
plt.axes().set_aspect('equal')
plt.plot([-ll,ll+2],[-ll,ll+2])
plt.savefig('./Figures/Figure2/qq_plot_ut'+gcm_name+'.svg')


plt.figure()
x=data_qq_up
qqplot(x[1],x[0], c='r', alpha=0.5, edgecolor='k')
qqplot(x[2],x[0],  c='g', alpha=0.5, edgecolor='k')
qqplot( x[3],x[0], c='b', alpha=0.5, edgecolor='k')
plt.legend(['GCM','BCSD-GCM','BCSD-Rnd'])
plt.ylim([0,80])
plt.xlim([0,80])
plt.plot([0,80],[0,80])

plt.xlabel('mm/month')
plt.ylabel('Observed data')
plt.axes().set_aspect('equal')
plt.savefig('./Figures/Figure2/qq_plot_up'+gcm_name+'.svg')


# othe

In [ ]:
import statsmodels.api as sm
x=data_qq_ip
pp_x = sm.ProbPlot(x[0])
pp_y = sm.ProbPlot(x[1])
ax=plt.subplot(111)
fig1 = pp_x.qqplot(other=pp_y, line='45',ax=ax,color='r',alpha=0.5)
pp_y = sm.ProbPlot(x[2])
fig2 = pp_x.qqplot(other=pp_y, line='45',ax=ax,color='g',alpha=0.5)
pp_y = sm.ProbPlot(x[3])
fig3 = pp_x.qqplot(other=pp_y, line='45',ax=ax,color='b',alpha=0.5)
plt.axes().set_aspect('equal')
plt.figure()

qqplot(x[1],x[0], c='r', alpha=0.5, edgecolor='k')
qqplot(x[2],x[0],  c='g', alpha=0.5, edgecolor='k')
qqplot(x[3],x[0], c='b', alpha=0.5, edgecolor='k')
plt.ylim([0,80])
plt.xlim([0,80])
plt.plot([0,80],[0,80])
plt.xlabel('X')
plt.ylabel('Y')
plt.axes().set_aspect('equal')


In [ ]:
import statsmodels.api as sm
x=data_qq_up
pp_x = sm.ProbPlot(x[0])
pp_y = sm.ProbPlot(x[1])
ax1=plt.subplot(1,3,1)
fig1 = pp_x.qqplot(other=pp_y, line='45',ax=ax1,color='r',alpha=0.5)
pp_y = sm.ProbPlot(x[2])
fig2 = pp_x.qqplot(other=pp_y, line='45',ax=ax1,color='g',alpha=0.5)
pp_y = sm.ProbPlot(x[3])
fig3 = pp_x.qqplot(other=pp_y, line='45',ax=ax1,color='b',alpha=0.5)
plt.axes().set_aspect('equal')

x=data_qq_ut
pp_x = sm.ProbPlot(x[0])
pp_y = sm.ProbPlot(x[1])
ax2=plt.subplot(132)
fig1 = pp_x.qqplot(other=pp_y, line='45',ax=ax2,color='r',alpha=0.5)
pp_y = sm.ProbPlot(x[2])
fig2 = pp_x.qqplot(other=pp_y, line='45',ax=ax2,color='g',alpha=0.5)
pp_y = sm.ProbPlot(x[3])
fig3 = pp_x.qqplot(other=pp_y, line='45',ax=ax2,color='b',alpha=0.5)
plt.axes().set_aspect('equal')


x=data_qq_up
pp_x = sm.ProbPlot(x[0])
pp_y = sm.ProbPlot(x[1])
ax3=plt.subplot(133)
fig1 = pp_x.qqplot(other=pp_y, line='45',ax=ax3,color='r',alpha=0.5)
pp_y = sm.ProbPlot(x[2])
fig2 = pp_x.qqplot(other=pp_y, line='45',ax=ax3,color='g',alpha=0.5)
pp_y = sm.ProbPlot(x[3])
fig3 = pp_x.qqplot(other=pp_y, line='45',ax=ax3,color='b',alpha=0.5)
plt.axes().set_aspect('equal')



In [ ]:
titles=['Obs',
        'GCM-M',
        'GCM-C',
        'GCM-H',
        'GCM-G',
        'BC-M',
        'BC-C',
        'BC-H',
        'BC-G',
        'BC-R0',
        'BC-R1',
        'BC-R2',
        'BC-R3']

In [ ]:
titlesy=['GCM','BC-G','BC-R']
titlesx=[['N','S','W','MW'],['N','S','W','MW'],['NW','C','NC','NE','NM','P']]
plot_hmap_fig2s([ind1h,ind2h,ind3h],[region_shapes_us,region_shapes_us,region_shapes_in],['NAME','NAME','State_Name'],[4,4,6],[[0,5],[0,25],[0,5]],titlesx,titlesy,".1f",'Reds')

### R10mm

In [ ]:
R10mmindic=spatial_ind(R10mm_annualy,data)
plot_indices(R10mmindic,titles,'R10mm','# of days',[0,100],[-15,15],ex='max')

In [ ]:
#R10mma,R10mmb=get_corr_n_rmse(R10mmindic)
plot_heatmap(R10mma,[-1,1],titles,".1f",'RdBu_r')
plot_heatmap(R10mmb,[0,100],titles,".0f",'Reds')

### CWD

In [ ]:
spatial_ind(CDD_annualy,'CWD','# of days',[0,100],[-10,10])

### Rx5day

In [ ]:
spatial_ind(Rx5day_annualy,'Rx5day','mm',[0,200],[-15,15])

### SDII

In [ ]:
spatial_ind(SDII_annualy,'SDII','mm',[0,100],[-2,2])

### R95pTOT

In [ ]:
spatial_ind(R95pTOT_annualy,'R95pTOT','%age',[0,30],[-5,5])